In [1]:
import openai
import pandas as pd
import re
import time

from tqdm import tqdm

В данном случае у нас есть текст ФЗ, который послужит базой для QA-системы. Разобьем его кусочки: глава-статья и построим для них эмбеддинги с помощью OpenAI API (*Эмбеддинги можно получить также с помощью BERT или любой другой открытой модели*).

In [2]:
with open('fz-126.txt', 'r') as file:
    law = file.read()

In [3]:
chapters_idx = re.finditer('ГЛАВА \d+.', law)
indices = [c.start(0) for c in chapters_idx]
# не знаю регулярок, обхожу как могу
chapters = dict([(law[i:].split('\n')[0], '\n'.join(law[i:].split('\n')[1:])) for i in indices]) 
chapters = {}

for i in indices:
    chapter = law[i:].split('\n')[0]
    chapters[chapter] = {'full_text': '\n'.join(law[i:].split('\n')[1:])}

In [4]:
chapters.keys()

dict_keys(['ГЛАВА 1.\xa0ОБЩИЕ ПОЛОЖЕНИЯ', 'ГЛАВА 2.\xa0ОСНОВЫ ДЕЯТЕЛЬНОСТИ В ОБЛАСТИ СВЯЗИ', 'ГЛАВА 3.\xa0СЕТИ СВЯЗИ', 'ГЛАВА 4.\xa0ПРИСОЕДИНЕНИЕ СЕТЕЙ ЭЛЕКТРОСВЯЗИ И ИХ ВЗАИМОДЕЙСТВИЕ', 'ГЛАВА 5.\xa0ГОСУДАРСТВЕННОЕ РЕГУЛИРОВАНИЕ ДЕЯТЕЛЬНОСТИ В ОБЛАСТИ СВЯЗИ', 'ГЛАВА 6.\xa0ЛИЦЕНЗИРОВАНИЕ ДЕЯТЕЛЬНОСТИ В ОБЛАСТИ ОКАЗАНИЯ УСЛУГ СВЯЗИ И ОЦЕНКА СООТВЕТСТВИЯ В ОБЛАСТИ СВЯЗИ\xa0', 'ГЛАВА 7.\xa0УСЛУГИ СВЯЗИ', 'ГЛАВА 8.\xa0УНИВЕРСАЛЬНЫЕ УСЛУГИ СВЯЗИ', 'ГЛАВА 9.\xa0ЗАЩИТА ПРАВ ПОЛЬЗОВАТЕЛЕЙ УСЛУГАМИ СВЯЗИ', 'ГЛАВА 10. УПРАВЛЕНИЕ СЕТЯМИ СВЯЗИ В ОТДЕЛЬНЫХ СЛУЧАЯХ', 'ГЛАВА 11.\xa0ОТВЕТСТВЕННОСТЬ ЗА НАРУШЕНИЕ ЗАКОНОДАТЕЛЬСТВА РОССИЙСКОЙ ФЕДЕРАЦИИ В ОБЛАСТИ СВЯЗИ', 'ГЛАВА 12.\xa0МЕЖДУНАРОДНОЕ СОТРУДНИЧЕСТВО РОССИЙСКОЙ ФЕДЕРАЦИИ В ОБЛАСТИ СВЯЗИ', 'ГЛАВА 13.\xa0ЗАКЛЮЧИТЕЛЬНЫЕ И ПЕРЕХОДНЫЕ ПОЛОЖЕНИЯ'])

In [5]:
for chapter in chapters.keys():
    text = chapters[chapter]['full_text']
    articles_idx = re.finditer('\nСтатья \d+\.', text)
    indices = [c.start(0)+1 for c in articles_idx]
    
    articles = {}
    for i in range(len(indices)-1):
        article_text = text[indices[i]:indices[i+1]]
        if len(article_text) >= 8191:
            article_text = article_text[:8191]
        articles[text[i:].split('\n')[0]] = article_text        
        
    chapters[chapter]['articles'] = articles

In [6]:
df = pd.DataFrame()

In [7]:
for chapter in chapters.keys():
    for article in chapters[chapter]['articles'].keys():
        article_text = chapters[chapter]['articles'][article]
        df = df.append({'chapter': chapter, 'article': article_text, 'embedding': None}, ignore_index=True)

/tmp/ipykernel_737367/2589238372.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'chapter': chapter, 'article': article_text, 'embedding': None}, ignore_index=True)
/tmp/ipykernel_737367/2589238372.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'chapter': chapter, 'article': article_text, 'embedding': None}, ignore_index=True)
/tmp/ipykernel_737367/2589238372.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'chapter': chapter, 'article': article_text, 'embedding': None}, ignore_index=True)
/tmp/ipykernel_737367/2589238372.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df =

In [8]:
df

,chapter,article,embedding
0,ГЛАВА 1. ОБЩИЕ ПОЛОЖЕНИЯ,Статья 49. Учетно-отчетное время в области свя...,None
1,ГЛАВА 1. ОБЩИЕ ПОЛОЖЕНИЯ,Статья 50. Служебная электросвязь\n \n1. Служе...,None
2,ГЛАВА 1. ОБЩИЕ ПОЛОЖЕНИЯ,Статья 3. Сфера действия настоящего Федерально...,None
3,ГЛАВА 1. ОБЩИЕ ПОЛОЖЕНИЯ,Статья 4. Законодательство Российской Федераци...,None
4,ГЛАВА 1. ОБЩИЕ ПОЛОЖЕНИЯ,Статья 5. Собственность на сети связи и средст...,None
...,...,...,...
414,ГЛАВА 12. МЕЖДУНАРОДНОЕ СОТРУДНИЧЕСТВО РОССИЙС...,Статья 70. Регулирование деятельности в област...,None
415,ГЛАВА 12. МЕЖДУНАРОДНОЕ СОТРУДНИЧЕСТВО РОССИЙС...,Статья 71. Ввоз радиоэлектронных средств и выс...,None
416,ГЛАВА 12. МЕЖДУНАРОДНОЕ СОТРУДНИЧЕСТВО РОССИЙС...,Статья 72. Международная почтовая связь\n \nАд...,None
417,ГЛАВА 12. МЕЖДУНАРОДНОЕ СОТРУДНИЧЕСТВО РОССИЙС...,Статья 73. Приведение законодательных актов в ...,None


In [9]:
openai.api_key = 'sk-xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'

In [10]:
def get_embedding(text):
    response = openai.Embedding.create(
        input=text,
        model="text-embedding-ada-002"
    )
    return response['data'][0]['embedding']

In [11]:
df['embedding']=df['embedding'].astype('object')

In [13]:
for index, row in tqdm(df.T.iteritems()):
    embedding = get_embedding(row.article)
    df.at[index, 'embedding'] = embedding
    time.sleep(2)

419it [21:38,  3.10s/it]


In [14]:
df

,chapter,article,embedding
0,ГЛАВА 1. ОБЩИЕ ПОЛОЖЕНИЯ,Статья 49. Учетно-отчетное время в области свя...,"[-0.005544021725654602, 0.008837137371301651, ..."
1,ГЛАВА 1. ОБЩИЕ ПОЛОЖЕНИЯ,Статья 50. Служебная электросвязь\n \n1. Служе...,"[-0.013055194169282913, 0.007684218231588602, ..."
2,ГЛАВА 1. ОБЩИЕ ПОЛОЖЕНИЯ,Статья 3. Сфера действия настоящего Федерально...,"[0.005478863138705492, 0.006418188102543354, 0..."
3,ГЛАВА 1. ОБЩИЕ ПОЛОЖЕНИЯ,Статья 4. Законодательство Российской Федераци...,"[0.011786730960011482, 0.009066236205399036, -..."
4,ГЛАВА 1. ОБЩИЕ ПОЛОЖЕНИЯ,Статья 5. Собственность на сети связи и средст...,"[0.0014680444728583097, 0.004267774987965822, ..."
...,...,...,...
414,ГЛАВА 12. МЕЖДУНАРОДНОЕ СОТРУДНИЧЕСТВО РОССИЙС...,Статья 70. Регулирование деятельности в област...,"[0.013144051656126976, 0.0011341888457536697, ..."
415,ГЛАВА 12. МЕЖДУНАРОДНОЕ СОТРУДНИЧЕСТВО РОССИЙС...,Статья 71. Ввоз радиоэлектронных средств и выс...,"[0.004452918190509081, -0.0018816750962287188,..."
416,ГЛАВА 12. МЕЖДУНАРОДНОЕ СОТРУДНИЧЕСТВО РОССИЙС...,Статья 72. Международная почтовая связь\n \nАд...,"[-0.0066809384152293205, 0.0037608793936669827..."
417,ГЛАВА 12. МЕЖДУНАРОДНОЕ СОТРУДНИЧЕСТВО РОССИЙС...,Статья 73. Приведение законодательных актов в ...,"[-0.00918667484074831, -0.0046682641841471195,..."


In [15]:
df.to_csv("fz-126.csv")